<a href="https://colab.research.google.com/github/remerge/uplift-report/blob/internal/uplift_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Remerge uplift report

This notebook allows you to validate remerge provided uplift reporting numbers. To do so it downloads and analyses exported campaign and event data from S3. The campaign data contains all users that remerge marked to be part of an uplift test, the A/B group assignment, the timestamp of marking, conversion events (click, app open or similar) and their cost. The event data reflects the app event stream and includes events, their timestamp and revenue (if any). We calculate the incremental revenue and the iROAS in line with the [remerge whitepaper](https://drive.google.com/file/d/1PTJ93Cpjw1BeiVns8dTcs2zDDWmmjpdc/view). 

**Hint**: This notebook can be run in any Jupyter instance with enough space/memory, as a [Google Colab notebook](#Google-Colab-version) or as a standalone Python script. If you are using a copy of this notebook running on Colab or locally you can find the original template on [GitHub: remerge/uplift-report](https://github.com/remerge/uplift-report/blob/internal/uplift_report_per_campaign.ipynb)

### Notebook configuration

For this notebook to work properly several variables in the [Configuration](#Configuration) section need to be be set: `customer`, `audience`, `
revenue_event`, `dates` and the AWS credentials. All of these will be provided by your remerge account manager. 

In [0]:
# Import remerge uplift-report library
import os

# if we are in jupyter environment - we have cloned the repo already and `lib` is available
# on Colab we need to clone the repo and enable the same loading path through a symlink
if not os.path.exists('lib'):
    !git clone --branch internal https://github.com/remerge/uplift-report.git
    !ln -s uplift-report/lib
    
    !pip install lib/
    
    # Since we could have upgraded some dependencies, that require restart of the kernel (specifically `pandas`),
    # it is safer to perform this restart now
    os.kill(os.getpid(), 9)    

## Import packages

This notebook/script needs our Uplift Report helper library, as long as several other dependencies it brings with it


## Load helpers

In [0]:
import os
import pandas as pd

import lib.helpers as helpers

from IPython.display import display  # so we can run this as script as well

## Version
Version of the analysis script corresponding to the methodology version in the whitepaper (Major + Minor version represent the whitepaper version, revision represents changes and fixes of the uplift report script).

In [0]:
display(helpers.__version__)

## Configuration

Set the customer name, audience and access credentials for the S3 bucket and path. Furthermore the event for which we want to evaluate the uplift needs to be set `revenue_event`.

In [0]:
# configure path and revenue event 
customer = ''
audiences = ['']
revenue_event = 'purchase'

# date range for the report
dates = pd.date_range(start='2019-01-01',end='2019-01-01')

# AWS credentials
os.environ["AWS_ACCESS_KEY_ID"] = ''
os.environ["AWS_SECRET_ACCESS_KEY"] = ''

# Configure the reporting output: 

# named groups that aggregate several campaigns
groups = {}

# show uplift results per campaign:
per_campaign_results = False

# base statistical calculations on unique converters instead of conversions
use_converters_for_significance = False

# enable deduplication heuristic for appsflyer
use_deduplication = False

## Load CSV data from S3

Load mark, spend and event data from S3. 

### IMPORTANT

**The event data is usually quite large (several GB) so this operation might take several minutes or hours to complete, depending on the size and connection.**

In [0]:
marks_and_spend_df = helpers.load_marks_and_spend_data(customer, audiences, dates)

2019-08-20 12:24:28.797241 start loading CSV for 1590_Shopee_TH_iOS marks_and_spend 2019-07-01 00:00:00
2019-08-20 12:24:30.277685 finished loading CSV for 01.07.2019 took 0:00:01.480487
2019-08-20 12:24:30.279101 caching local as parquet cache-v5.4.15/1590_Shopee_TH_iOS/marks_and_spend/20190701.parquet
2019-08-20 12:24:30.290634 caching on S3 as parquet s3://remerge-customers/shopee/uplift_data/1590_Shopee_TH_iOS/marks_and_spend/cache-v5.4.15/20190701.parquet
2019-08-20 12:24:31.283613 start loading CSV for 1590_Shopee_TH_iOS marks_and_spend 2019-07-02 00:00:00
2019-08-20 12:24:32.754970 finished loading CSV for 02.07.2019 took 0:00:01.471391
2019-08-20 12:24:32.756401 caching local as parquet cache-v5.4.15/1590_Shopee_TH_iOS/marks_and_spend/20190702.parquet
2019-08-20 12:24:32.767365 caching on S3 as parquet s3://remerge-customers/shopee/uplift_data/1590_Shopee_TH_iOS/marks_and_spend/cache-v5.4.15/20190702.parquet
2019-08-20 12:24:33.781141 start loading CSV for 1590_Shopee_TH_iOS ma

In [0]:
attributions_df = helpers.load_attribution_data(customer, audiences, dates, revenue_event, marks_and_spend_df, use_deduplication)

Print some statistics of the loaded data sets.

In [0]:
marks_and_spend_df.info(memory_usage='deep')

In [0]:
attributions_df.info(memory_usage='deep')

### Calculate and display uplift report for the data set as a whole

This takes the whole data set and calculates uplift KPIs.

In [0]:
report = helpers.uplift_report(marks_and_spend_df, attributions_df, groups, per_campaign_results, use_converters_for_significance)

## Uplift Results

You can configure the ouput by using variables in the 'Configuration' section

In [0]:
# set formatting options
pd.set_option('display.float_format', '{:.5f}'.format)

In [0]:
display(report)

## Export to reports overview

In [0]:
 helpers.export_to_overview(customer, audiences, dates, report)

### CSV Export - combined reports

In [0]:
start = dates[0]
end = dates[-1]
helpers.export_csv(report,'{}_{}-{}.csv'.format(customer, start, end))